### Import Library

In [1]:
!pip install gym==0.17.3 --quiet
import numpy as np
import gym
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.25.0 requires cloudpickle>=2.0.0, but you have cloudpickle 1.6.0 which is incompatible.
dask 2024.10.0 requires cloudpickle>=3.0.0, but you have cloudpickle 1.6.0 which is incompatible.


### Membuat Peta

In [2]:
# Membuat daftar peta
peta = [
    ['SFFF','FHFH','FFFH','HFFG'],
    ['SFFF','FFHF','HFFF','HFFG'],
    ['SHFF','FHFH','FFFH','HHFG'],
    ['SFFF','HHFF','FFFF','HFFG'],
    ['SFFH','FFFH','HFFH','HHFG']
]

In [15]:
peta[0]

['SFFF', 'FHFH', 'FFFH', 'HFFG']

In [4]:
# Memuat lingkungan
env = gym.make("FrozenLake-v0",is_slippery=False, desc=peta[0])

In [5]:
n_observations = env.observation_space.n
n_actions      = env.action_space.n

print('Banyak State  : ' + str(n_observations))
print('Banyak Action : ' + str(n_actions))

Banyak State  : 16
Banyak Action : 4


In [10]:
ACTION = ["KIRI","BAWAH","KANAN","ATAS"]

In [11]:
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


### Menguji Langkah Agen

In [12]:
# Langkah 1 (ke Kanan)
new_state, reward, done, info = env.step(2)

# Menampilkan informasi
print(f"New State : {new_state}")
print(f"Reward    : {reward}")
print(f"Done      : {done}")

# Menampilkan visualisasi lingkungan
env.render()

New State : 1
Reward    : 0.0
Done      : False
  (Right)
SFFF
FHFH
FFFH
HFFG


In [13]:
# Langkah 2 (ke Kanan)
new_state, reward, done, info = env.step(2)

# Menampilkan informasi
print(f"New State : {new_state}")
print(f"Reward    : {reward}")
print(f"Done      : {done}")

# Menampilkan visualisasi lingkungan
env.render()

New State : 2
Reward    : 0.0
Done      : False
  (Right)
SFFF
FHFH
FFFH
HFFG


In [14]:
# Langkah 3 (ke Bawah)
new_state, reward, done, info = env.step(1)

# Menampilkan informasi
print('New State : {}'.format(new_state))
print('Reward    : {}'.format(reward))
print('Done      : {}'.format(done))

# Menampilkan visualisasi lingkungan
env.render()

New State : 6
Reward    : 0.0
Done      : False
  (Down)
SFFF
FHFH
FFFH
HFFG


In [16]:
# Langkah 4 (ke Bawah)
new_state, reward, done, info = env.step(1)

# Menampilkan informasi
print('New State : {}'.format(new_state))
print('Reward    : {}'.format(reward))
print('Done      : {}'.format(done))

# Menampilkan visualisasi lingkungan
env.render()

New State : 10
Reward    : 0.0
Done      : False
  (Down)
SFFF
FHFH
FFFH
HFFG


In [17]:
# Langkah 5 (ke Bawah)
new_state, reward, done, info = env.step(1)

# Menampilkan informasi
print(f"New State : {new_state}")
print(f"Reward    : {reward}")
print(f"Done      : {done}")

# Menampilkan visualisasi lingkungan
env.render()

New State : 14
Reward    : 0.0
Done      : False
  (Down)
SFFF
FHFH
FFFH
HFFG


In [18]:
# Langkah 6 (ke Kanan)
new_state, reward, done, info = env.step(2)

# Menampilkan informasi
print(f"New State : {new_state}")
print(f"Reward    : {reward}")
print(f"Done      : {done}")

# Menampilkan visualisasi lingkungan
env.render()

New State : 15
Reward    : 1.0
Done      : True
  (Right)
SFFF
FHFH
FFFH
HFFG


### Training Model

In [19]:
def train_agent(env, n_episodes=10000, max_iter_episode=100, exploration_proba=1, exploration_decreasing_decay=0.001, min_exploration_proba=0.01, gamma=0.99, lr=0.1):
    # Inisialisasi Q-table dengan ukuran berdasarkan jumlah state dan aksi
    Q_table = np.zeros((env.observation_space.n, env.action_space.n))

    # List untuk menyimpan reward dari setiap episode
    rewards_per_episode = []

    # Loop melalui setiap episode
    for episode in range(n_episodes):
        # Reset lingkungan untuk memulai episode baru dan mendapatkan state awal
        state = env.reset()

        # Inisialisasi total reward episode menjadi 0
        episode_reward = 0

        # Loop melalui setiap iterasi dalam episode
        for _ in range(max_iter_episode):
            # Pilih tindakan berdasarkan probabilitas eksplorasi atau menggunakan kebijakan Q
            if np.random.uniform(0, 1) < exploration_proba:
                action = env.action_space.sample()  # Aksi acak (eksplorasi)
            else:
                action = np.argmax(Q_table[state, :])  # Aksi terbaik berdasarkan Q-table (eksploitasi)

            # Ambil langkah berdasarkan tindakan yang dipilih
            next_state, reward, done, _ = env.step(action)

            # Update Q-value berdasarkan reward yang diterima dan perkiraan nilai Q di state berikutnya
            Q_table[state, action] = (1 - lr) * Q_table[state, action] + lr * (reward + gamma * np.max(Q_table[next_state, :]))

            # Tambahkan reward dari langkah ini ke total reward episode
            episode_reward += reward
            state = next_state  # Pindah ke state berikutnya

            # Hentikan episode jika mencapai terminal state
            if done:
                break

        # Kurangi probabilitas eksplorasi seiring berjalannya waktu
        exploration_proba = max(min_exploration_proba, np.exp(-exploration_decreasing_decay * episode))

        # Simpan total reward episode ke dalam list
        rewards_per_episode.append(episode_reward)

    # Cetak rata-rata reward per 1000 episode
    print("Rata-Rata Reward per 1000 Episode")
    for i in range(10):
        print((i + 1) * 1000, " : Rata-Rata Reward: ", np.mean(rewards_per_episode[1000 * i:1000 * (i + 1)]))

    # Kembalikan Q-table yang telah dilatih
    return Q_table

In [22]:
Q_table_all = []  # List kosong untuk menyimpan semua tabel Q dari setiap lingkungan

for peta_env in peta:  # Loop melalui setiap peta dalam daftar peta
    # Load Environment untuk setiap peta dengan konfigurasi tertentu
    env = gym.make("FrozenLake-v0", is_slippery=False, desc=peta_env)
    env.reset()  # Reset lingkungan ke keadaan awal

    print('Peta : ')
    print(peta_env)  # Cetak peta yang sedang digunakan

    # Melatih Agent pada lingkungan saat ini
    Q_table = train_agent(env)  # Panggil fungsi train_agent untuk melatih agen di lingkungan saat ini

    # Menyimpan Q_table untuk lingkungan saat ini ke dalam list
    Q_table_all.append(Q_table)  # Tambahkan Q_table ke dalam list Q_table_all

    print()  # Cetak baris kosong untuk pemisah antara lingkungan yang berbeda


Peta : 
['SFFF', 'FHFH', 'FFFH', 'HFFG']
Rata-Rata Reward per 1000 Episode
1000  : Rata-Rata Reward:  0.276
2000  : Rata-Rata Reward:  0.729
3000  : Rata-Rata Reward:  0.911
4000  : Rata-Rata Reward:  0.967
5000  : Rata-Rata Reward:  0.993
6000  : Rata-Rata Reward:  0.991
7000  : Rata-Rata Reward:  0.988
8000  : Rata-Rata Reward:  0.99
9000  : Rata-Rata Reward:  0.979
10000  : Rata-Rata Reward:  0.989

Peta : 
['SFFF', 'FFHF', 'HFFF', 'HFFG']
Rata-Rata Reward per 1000 Episode
1000  : Rata-Rata Reward:  0.292
2000  : Rata-Rata Reward:  0.778
3000  : Rata-Rata Reward:  0.913
4000  : Rata-Rata Reward:  0.965
5000  : Rata-Rata Reward:  0.992
6000  : Rata-Rata Reward:  0.992
7000  : Rata-Rata Reward:  0.991
8000  : Rata-Rata Reward:  0.993
9000  : Rata-Rata Reward:  0.987
10000  : Rata-Rata Reward:  0.991

Peta : 
['SHFF', 'FHFH', 'FFFH', 'HHFG']
Rata-Rata Reward per 1000 Episode
1000  : Rata-Rata Reward:  0.0
2000  : Rata-Rata Reward:  0.0
3000  : Rata-Rata Reward:  0.0
4000  : Rata-Rata R

In [21]:
Q_table_all

[array([[0.94148015, 0.95099005, 0.93206535, 0.94148015],
        [0.94148015, 0.        , 0.74063515, 0.87055359],
        [0.87660457, 0.21674049, 0.0517229 , 0.04142206],
        [0.22374726, 0.        , 0.00917317, 0.00828988],
        [0.95099005, 0.96059601, 0.        , 0.94148015],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.86976285, 0.        , 0.11154469],
        [0.        , 0.        , 0.        , 0.        ],
        [0.960596  , 0.        , 0.970299  , 0.95099005],
        [0.96059587, 0.9801    , 0.98009995, 0.        ],
        [0.84404355, 0.99      , 0.        , 0.63373197],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.98009989, 0.99      , 0.97029887],
        [0.98009821, 0.98999977, 1.        , 0.98009575],
        [0.        , 0.        , 0.        , 0.        ]]),
 array([[0.94148015, 0.95099005, 0.95099005, 0.94148015],
        [0.9

### Memainkan Agen yang Telah Dilatih


In [23]:
index_peta = 0 # silakan pilih peta

env = gym.make("FrozenLake-v0",is_slippery=False, desc=peta[index_peta])
env.reset()

0

In [24]:
for langkah in range(1, 7):
    if langkah == 1:
        best_action = np.argmax(Q_table_all[index_peta][0])
    else:
        best_action = np.argmax(Q_table_all[index_peta][current_state])

    new_state, reward, done, info = env.step(best_action)

    # Cetak informasi langkah
    print('--------------------------------------')
    print('Langkah ke  :', langkah)
    print('Best Action :', ACTION[best_action])
    print('New State   :', new_state)
    print('Reward      :', reward)
    print('Done        :', done)

    # Tampilkan visualisasi lingkungan
    env.render()
    current_state = new_state  # Perbarui state saat ini

    print()

--------------------------------------
Langkah ke  : 1
Best Action : KANAN
New State   : 1
Reward      : 0.0
Done        : False
  (Right)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 2
Best Action : KANAN
New State   : 2
Reward      : 0.0
Done        : False
  (Right)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 3
Best Action : BAWAH
New State   : 6
Reward      : 0.0
Done        : False
  (Down)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 4
Best Action : BAWAH
New State   : 10
Reward      : 0.0
Done        : False
  (Down)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 5
Best Action : BAWAH
New State   : 14
Reward      : 0.0
Done        : False
  (Down)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 6
Best Action : KANAN
New State   : 15
Reward      : 1.0
Done        : True
  (Right)
SFFF
FHFH
FFFH
HFFG



### Evaluasi

In [25]:
for peta_env in peta:  # Iterasi melalui setiap peta dalam daftar peta
    print("Peta   :", peta_env)  # Cetak deskripsi peta yang sedang diperiksa

    env = gym.make("FrozenLake-v0", is_slippery=False, desc=peta_env)  # Buat lingkungan permainan Frozen Lake
    env.reset()  # Atur ulang lingkungan ke keadaan awal

    # Iterasi untuk agen melakukan langkah-langkah dalam lingkungan
    for langkah in range(1, 7):
        if langkah == 1:
            best_action = np.argmax(Q_table_all[index_peta][0])  # Ambil tindakan terbaik untuk langkah pertama
        else:
            best_action = np.argmax(Q_table_all[index_peta][current_state])  # Ambil tindakan terbaik berdasarkan state saat ini

        new_state, reward, done, info = env.step(best_action)  # Lakukan langkah terbaik dalam lingkungan

        current_state = new_state  # Perbarui state saat ini

    # Periksa apakah agen berhasil menyelesaikan permainan
    if done:
        print("Status : Agent dapat menyelesaikan peta ini")  # Cetak pesan jika agen berhasil menyelesaikan permainan
    else:
        print("Status : Agent tidak dapat menyelesaikan peta ini")  # Cetak pesan jika agen gagal menyelesaikan permainan

    print()  # Cetak baris kosong sebagai pemisah antara hasil dari setiap peta

Peta   : ['SFFF', 'FHFH', 'FFFH', 'HFFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SFFF', 'FFHF', 'HFFF', 'HFFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SHFF', 'FHFH', 'FFFH', 'HHFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SFFF', 'HHFF', 'FFFF', 'HFFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SFFH', 'FFFH', 'HFFH', 'HHFG']
Status : Agent dapat menyelesaikan peta ini



### Menyimpan Model

In [26]:
# Simpan tabel Q yang telah dilatih menggunakan modul pickle
pickle.dump(Q_table_all, open('Q_table_Frozen_Lake.model', 'wb'))

# Penjelasan penutup
print("Tabel Q untuk lingkungan Frozen Lake telah berhasil disimpan dalam file 'Q_table_Frozen_Lake.model'. Proses pelatihan agen dan penyimpanan tabel Q ini memungkinkan untuk digunakan kembali dalam pengujian atau aplikasi selanjutnya tanpa perlu melakukan pelatihan ulang.")

Tabel Q untuk lingkungan Frozen Lake telah berhasil disimpan dalam file 'Q_table_Frozen_Lake.model'. Proses pelatihan agen dan penyimpanan tabel Q ini memungkinkan untuk digunakan kembali dalam pengujian atau aplikasi selanjutnya tanpa perlu melakukan pelatihan ulang.
